In [1]:
import time

import numpy as np
import pandas as pd

from preprocessing.preprocessing import build_dataset

pd.options.mode.chained_assignment = None 

In [2]:
def detect_group_leakage(df, group_name, split_name):
    
    df_overlapping = df.groupby([group_name])[split_name].nunique()
    return any(df_overlapping>1)

def plot_split_size(df, split_name, labels_names):
    
    df_plot = df.groupby([split_name])[labels_names].sum()
    display(round(df_plot/df_plot.sum(),2))
    

In [3]:
# Multiclass
wl = 3
sr = 22050
flims = (1,10000)
site_list = ['INCT20955','INCT41']
path_save ='data/BuildDataset/datasetv2-multiclass_1'
labels_cols = ['BOAFAB_F' ,'BOAFAB_M', 'PHYCUV_F' ,'PHYCUV_M', 'BOAFAB_C' ,'DENCRU_M',
                 'BOALUN_M', 'BOAALB_M', 'BOAALB_F' ,'DENCRU_F' ,'BOALUN_F', 'PHYMAR_F',
                 'PHYMAR_M' ,'PITAZU_M' ,'PITAZU_F' ,'BOALUN_C' ,'PHYMAR_C']
start = time.time()
df_multiclass = build_dataset(wl=wl, 
                              site_list=site_list,
                              target_sr=sr, 
                              flims=flims,
                              path_save=path_save,
                             labels_cols=labels_cols)
end = time.time()
print(round((end - start)/60,1),' minutes')

3023 327 2696
(6158, 6)
3999 1289 2710
(2540, 6)


/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-baseline/code/Users/jscanass/chorus_experiments/preprocessing/preprocessing.py:258: UserWarning: We want 2710samples but only have2540
  warn('We want ' +str(n_sample)+ 'samples but only have'+ str(df_absence_rois_from_planilha.shape[0]))
/anaconda/envs/trans_env/lib/python3.8/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


3.6  minutes


In [10]:
df_dataset_concat = df_multiclass.copy()
df_dataset_concat.shape

(13874, 33)

In [11]:
pd.crosstab(df_dataset_concat['subset'], df_dataset_concat['fold'])

fold,0.0,1.0,2.0,3.0,4.0,5.0
subset,,,,,,
test,2303,0,0,0,0,0
train,0,2298,2309,2334,2315,2315


In [12]:
print('Group Leakage:', detect_group_leakage(df=df_dataset_concat, 
                 group_name='fname',
                 split_name='fold'))

Group Leakage: False


In [13]:
df_dataset_concat_D = df_dataset_concat.copy()
df_dataset_concat_D['dummy'] = 1
df_dataset_concat_D = df_dataset_concat_D.pivot_table(index='column_name', columns = 'fold', values = 'dummy',aggfunc=np.sum)
df_dataset_concat_D = pd.concat([df_dataset_concat_D,
                                df_dataset_concat['column_name'].value_counts()],axis=1,)
df_dataset_concat_D= pd.concat([df_dataset_concat_D,
                                pd.DataFrame(df_dataset_concat['fold'].value_counts(normalize=False)).T])
df_dataset_concat_D

,0.0,1.0,2.0,3.0,4.0,5.0,column_name
INCT20955_ABSENCE,503.0,504.0,505.0,505.0,503.0,503.0,3023.0
INCT20955_BOAFAB_C,NaN,NaN,20.0,20.0,20.0,20.0,80.0
INCT20955_BOAFAB_F,80.0,76.0,78.0,84.0,75.0,74.0,467.0
INCT20955_BOAFAB_M,247.0,246.0,244.0,248.0,247.0,254.0,1486.0
INCT20955_PHYCUV_F,141.0,137.0,134.0,134.0,138.0,136.0,820.0
INCT20955_PHYCUV_M,30.0,30.0,33.0,27.0,25.0,25.0,170.0
INCT41_ABSENCE,638.0,642.0,636.0,640.0,638.0,635.0,3829.0
INCT41_BOAALB_F,161.0,161.0,163.0,160.0,161.0,161.0,967.0
INCT41_BOAALB_M,113.0,111.0,109.0,111.0,112.0,109.0,665.0
INCT41_BOALUN_C,NaN,NaN,NaN,14.0,NaN,NaN,14.0


In [19]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT20955_BOAFAB_C']['fname'].unique()

array(['INCT20955_20191031_190000', 'INCT20955_20191031_200000',
       'INCT20955_20191107_191500', 'INCT20955_20191122_201500'],
      dtype=object)

In [20]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT41_BOALUN_C']['fname'].unique()

array(['INCT41_20201028_234500'], dtype=object)

In [21]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT41_DENCRU_M']['fname'].unique()

array(['INCT41_20200126_190000', 'INCT41_20200216_200000',
       'INCT41_20200216_210000', 'INCT41_20200216_230000',
       'INCT41_20200218_191500'], dtype=object)

In [22]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT41_PHYMAR_C']['fname'].unique()

array(['INCT41_20201220_003000', 'INCT41_20201220_013000',
       'INCT41_20201220_023000', 'INCT41_20201220_043000'], dtype=object)

In [23]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT41_PHYMAR_M']['fname'].unique()

array(['INCT41_20201015_223000', 'INCT41_20201220_053000',
       'INCT41_20201220_193000'], dtype=object)

In [24]:
df_dataset_concat[df_dataset_concat['column_name']=='INCT41_PITAZU_M']['fname'].unique()

array(['INCT41_20201021_000000', 'INCT41_20201220_023000'], dtype=object)

In [ ]:
# Binary BOAFAB
wl = 3
sr = 22050
flims = (100,2000)
verbose = False # Doubt about use this term
recordings_folder = 'data/INCT20955/raw/recordings/'
annotation_path = 'data/INCT20955/raw/annotations/'
path_save = 'data/datasetv1/binary_boafab_1'
prefix='SAMPLE_'
labels_cols = ['BOAFAB_M','BOAFAB_C'] # looks like multiclass but ends like binary
site=['INCT20955']

start = time.time()
df_boafab_binary_dataset = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')

In [ ]:
# Binary PHYCUV
wl = 3
sr = 22050
flims = (300,2000)
verbose=True
recordings_folder = 'data/INCT20955/raw/recordings/'
annotation_path = 'data/INCT20955/raw/annotations/'
path_save='data/datasetv1/binary_phycuv_1'
prefix='SAMPLE_'
labels_cols = ['PHYCUV_M']
site=['INCT20955']

start = time.time()
df_phycuv_binary_dataset = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')

In [ ]:
# Multiclass
wl = 3
sr = 22050
flims = (1,10000)
verbose = False # Doubt about use this term
recordings_folder = 'data/INCT41/raw/recordings/'
annotation_path = 'data/INCT41/raw/annotations/'
path_save='data/datasetv2/multiclass_1'
prefix='SAMPLE_'
max_duration = 60
labels_cols = ['BOAFAB_M','BOAFAB_C','PHYCUV_M']
site=['INCT20955','INCT4']

start = time.time()
df_boafab_multiclass_1 = build_dataset(wav_path=recordings_folder, 
                                                  annotation_path=annotation_path, 
                                                  wl=wl, 
                                                  site=site,
                                                  target_sr=sr, 
                                                  flims=flims,
                                                  path_save=path_save, 
                                                  prefix=prefix, 
                                                  labels_cols=labels_cols,
                                                  verbose=verbose)
end = time.time()
print(round((end - start)/60,1),' minutes')